In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import sklearn as sk
import os
import random
import math
import datetime


In [2]:
tf.test.is_built_with_cuda()

True

# Data Pipeline

Extract video frames

In [3]:
IMG_SIZE = 224

v1 = "video_data/violence_dataset/Violence"
v2 = "video_data/surveillance_dataset/fight"

n1 = "video_data/violence_dataset/NonViolence"
n2 = "video_data/surveillance_dataset/nofight"
violent_videos = [v1, v2]
nonviolent_videos = [n1, n2]


NEW_FOLDER = "new_data_dir"

nonviolent_path = "video_data/new_data_dir/NonViolent"
violent_path = "video_data/new_data_dir/Violent"
''' make the list of paths to the videos
    extract the frames from the videos into images (what if i just extract two frames from each?)
    save these images in a new folder
    this folder is now your data dir
    make sure that each frame is in its respective folder (violence, non violence)'''

' make the list of paths to the videos\n    extract the frames from the videos into images (what if i just extract two frames from each?)\n    save these images in a new folder\n    this folder is now your data dir\n    make sure that each frame is in its respective folder (violence, non violence)'

we have video_to_frames(path) function, what I need to do is use [violent_videos, nonviolent_videos] to automatically go into NEW_FOLDER accordingly their new folders with the frames extracted already

so change video_to_frames(list) to take in a list of lists or strings

video_to_frames functions, takes in a path and extracts frames to outpath

In [13]:
def video_to_frames(PARENT_PATHS: list[str], out_path:str):
    count = 0
    for path in PARENT_PATHS: # path is already the needed directory, just walk through it for each video
    
            
        assert os.path.exists(path)
        
        for i, video in tqdm(enumerate(os.listdir(path)), total=len(os.listdir(path))):

            
            #video_to_frames(path + '/' + video, out_path)

            video = os.path.join(path, video)
            assert os.path.exists(video)
            

            cap = cv2.VideoCapture(video)
            assert cap

            assert cap.isOpened()

            while cap.isOpened():
                
                success, image = cap.read()
                ID = int(cap.get(1))

                n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                mid = int(n_frames/2)
                first_q = int(n_frames/4)
                third_q = int(n_frames * .75)
                end = n_frames
                
                #assert success

                if success:

                
                    if ID == first_q or ID == mid or ID == third_q or ID == end:
                        
                        
                        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        

                        frame_name = 'Frame'+str(count)+'.jpg'
                        cv2.imwrite(os.path.join(out_path, frame_name), image)
                        count+=1

                    else: continue
        
                    

                else: 
                    break
            cap.release()
            

In [14]:
video_to_frames(nonviolent_videos, nonviolent_path)

100%|██████████| 150/150 [00:11<00:00, 12.74it/s]


In [15]:
video_to_frames(violent_videos, violent_path)

100%|██████████| 150/150 [00:09<00:00, 15.62it/s]


BENCHMARK: preproccessed 2300 videos to images in less 60s
update: 58s to 15+18 s

extracting frames from each path and placing them in their new respective directory

In [17]:
print(f"length of non violent: {len(os.listdir(nonviolent_path))}, length of violent: {len(os.listdir(violent_path))} ")

length of non violent: 4600, length of violent: 4600 


This one below, keeping for x,y data lists to np array

In [ ]:
'''CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
# this is for using the video_to_frames to extract the frames into a list
for category in os.listdir(PATH):
    
    
    path = os.path.join(PATH, category)
    #print(path)
    #assert os.path.exists(path)


    class_num = CATEGORIES.index(category)
    for i, video in tqdm(enumerate(os.listdir(path)), total=len(os.listdir(path))):

        #print(path + '/' + video)
        #print(os.path.join(path, video))
        frames = video_to_frames(path + '/' + video)
        if frames:
        
            for j, frame in enumerate(frames):
                X_set.append(frame)
                y_set.append(class_num)
        else: continue

            
print("success")'''

In [ ]:
X_set = np.array(X_set).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_set = np.array(y_set)
len(X_set), len(y_set)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.2)

for train_index, test_index in stratified_sample.split(X_set, y_set):
    X_train, X_test = X_set[train_index], X_set[test_index]
    y_train, y_test = y_set[train_index], y_set[test_index]

In [ ]:
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
#X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

y_train = np.asarray(y_train)
y_test_nn = np.asarray(y_test)
#y_val = np.asarray(y_val)

In [ ]:
from tensorflow.keras.utils import Sequence


class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X_train, y_train, 4)
test_gen = DataGenerator(X_test_nn, y_test_nn, 4)
#val_gen = DataGenerator(X_val, y_val, 16)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
from tensorflow.keras import Sequential, layers


data_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation"
)

## Building the Model

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.applications.mobilenet import MobileNet 
class OwnEfficientNetB0:
    @staticmethod
    def build(input_shape, data_augmentation, trainable=False, dropout=0.7):
        inputs = keras.Input(shape=input_shape)
        x = data_augmentation(inputs)
        x = preprocess_input(x)

        baseModel = MobileNet(weights="imagenet", include_top=False, input_tensor=x)
        baseModel.trainable = trainable

        headModel = baseModel.output
        headModel = layers.GlobalAveragePooling2D()(headModel)
        headModel = layers.Dropout(dropout)(headModel)
        outputs = layers.Dense(1, activation="sigmoid")(headModel)
        model = Model(inputs, outputs)

        return model

In [ ]:
model = OwnEfficientNetB0.build(input_shape=(IMG_SIZE,IMG_SIZE) + (3,), data_augmentation=data_augmentation)

In [ ]:
model.summary()

Compile the model

In [ ]:
#keep batchlayers intact, reduces learning time
for layer in model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=keras.metrics.BinaryAccuracy())

Fit the model

In [ ]:
from keras.callbacks import EarlyStopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.0001)]

In [ ]:
history = model.fit(
    train_gen, epochs=25, callbacks=callbacks
)


# Fine Tuning